# Undoing commits and changes


## Reviewing old commits

Beforing turning back the clock, we have to find out the point in the timeline that things went wrong. Review the commit history by
```bash
git log --oneline
```

If we only want to view a particular branch, do
```bash
git log <branch_name>
```

If we want to review changes across all branches,
```bash
git log --branches=*
```

## Checking out an older revision

To revert an older commit, run
```bash
git checkout <commit_id>
```
During the normal course of development, the **HEAD** usually points to master or some other local branch, but when you check out a previous commit, HEAD no longer points to a branch — it points directly to a commit. This is called a “**detached HEAD**” state.

The point is, your development should always take place on a branch—never on a detached HEAD. This makes sure you always have a reference to your new commits. However, if you’re just looking at an old commit, it doesn’t really matter if you’re in a detached HEAD state or not.

On the other hand, **checking out an old file** 
```bash
git checkout <commit_id> <filename>
git checkout <branch> <filename>
```
**does not move the HEAD pointer**. It remains on the same branch and same commit, avoiding a 'detached head' state. You can then commit the old version of the file in a new snapshot as you would any other changes. So, in effect, this usage of git checkout on a file, serves as a way to revert back to an old version of an individual file.

## Viewing an old revision
